Here we generate code to show how errors go down as sample size increases, doubling until 1 million.

In [ ]:
# Julia script: 1D Ising sampling + pseudolikelihood estimation

using Random
using Optim
using Statistics

# Gibbs sampler for 1D Ising with periodic boundary conditions
function gibbs_ising_1d(J, h, L, n_sweeps)
    # initialize spins randomly in {-1, +1}
    spins = rand(Bool, L)
    spins = [b ? Int8(1) : Int8(-1) for b in spins]
    for _ in 1:n_sweeps
        for i in 1:L
            # neighbors with periodic BC
            left = spins[mod1(i-1, L)]
            right = spins[mod1(i+1, L)]
            # local field
            m = J*(left + right) + h
            p = 1 / (1 + exp(-2*m))
            spins[i] = rand() < p ? Int8(1) : Int8(-1)
        end
    end
    return spins
end

# Generate M independent samples (after burn-in)
function generate_samples(J, h; L=50, burn_in=100, gap=10, M=1000)
    samples = Array{Int8}(undef, L, M)
    # initial burn-in
    state = gibbs_ising_1d(J, h, L, burn_in)
    for m in 1:M
        state = gibbs_ising_1d(J, h, L, gap)
        samples[:, m] = state
    end
    return samples
end

# Negative log-pseudolikelihood
function neglogpl(params, samples)
    J, h = params
    L, M = size(samples)
    total = 0.0
    for m in 1:M, i in 1:L
        s_i = samples[i, m]
        left = samples[mod1(i-1, L), m]
        right = samples[mod1(i+1, L), m]
        θ = J*(left + right) + h
        total += log(1 + exp(-2*s_i*θ))
    end
    return total
end

# Estimate (J,h) via pseudolikelihood
function estimate_pl(samples; init=(0.1, 0.0))
    result = optimize(p -> neglogpl(p, samples), collect(init), NelderMead())
    return Optim.minimizer(result)
end

# Main loop: vary sample size, repeat reps
function study(J_true, h_true; L=50, sizes=[40000], reps=10)
    results = Dict{Int, Array{Float64,2}}()
    for M in sizes
        println("\nSample size: $M")
        ests = zeros(reps, 2)
        for r in 1:reps
            samples = generate_samples(J_true, h_true; L=L, M=M)
            ests[r, :] = estimate_pl(samples)
            println("  rep $r: est_J=$(round(ests[r,1], digits=4)), est_h=$(round(ests[r,2], digits=4))")
        end
        results[M] = ests
    end
    return results
end

# Define your true parameters
J_true = 0.3
h_true = 0.1

# define sample sizes from 40k to 1e6 doubling
sizes = Int[]
M = 10000
while M <= 2_000_000
    push!(sizes, M)
    M *= 2
end


# run study
#default_results = study(J_true, h_true; L=4, sizes=sizes, reps=10)


In [ ]:
mae_results = Dict{Int, Tuple{Float64, Float64}}()
for (M, ests) in default_results
    mae_results[M] = (
    mean(abs.(ests[:,1] .- J_true)),
    mean(abs.(ests[:,2] .- h_true)))
end
println("MAE from default_results: ", mae_results)

In [ ]:
sizes_sorted = sort(collect(keys(mae_results)))
mae_J = [mae_results[s][1] for s in sizes_sorted]
mae_h = [mae_results[s][2] for s in sizes_sorted]

# 3) (Optional) Print or plot them
println("Sample sizes: ", sizes_sorted)
println("MAE for J:    ", mae_J)
println("MAE for h:    ", mae_h)

In [ ]:
coef_J = mae_J[1] * sqrt(sizes_sorted[1])
ref_J = [coef_J / sqrt(n) for n in sizes_sorted]
coef_h = mae_h[1] * sqrt(sizes_sorted[1])
ref_h = [coef_h / sqrt(n) for n in sizes_sorted]

In [ ]:
Now we study how errors go down for different values of coupling $J$ versus sample sizes with error bars. $J_1 = 0.2$ and $J_2 = 2.0$ with $h = 0.1$ in both cases.

In [ ]:
using Printf
function study_errors(J_true, h_true; L=4, sizes=[40000], reps=10)
    mean_abs_J_errors = Float64[]
    std_abs_J_errors = Float64[]
    for M in sizes
        J_errs = Float64[]
        for r in 1:reps
            samples = generate_samples(J_true, h_true; L=L, M=M)
            J_est, h_est = estimate_pl(samples)
            push!(J_errs, abs(J_est - J_true))
        end
        m_err = mean(J_errs)
        s_err = std(J_errs)
        @printf("N = %7d: mean |Ĵ–J₀| = %.3e, std = %.3e\n", M, m_err, s_err)
        push!(mean_abs_J_errors, m_err)
        push!(std_abs_J_errors, s_err)
    end
    return mean_abs_J_errors, std_abs_J_errors
end

# Set parameters
h_true = 0.1
sizes = Int[]
M = 10000
while M <= 2_000_000
    push!(sizes, M)
    M *= 2
end

# For J = 0.2
J_true1 = 0.2
println("=== J = 0.2 ===")
mean_J1, std_J1 = study_errors(J_true1, h_true; L=5, sizes=sizes, reps=100)

# For J = 2.0
J_true2 = 2.0
println("\n=== J = 2.0 ===")
mean_J2, std_J2 = study_errors(J_true2, h_true; L=5, sizes=sizes, reps=100)

# Print or save arrays for later use
println("\nMean abs errors for J=0.2: ", mean_J1)
println("Std abs errors for J=0.2: ", std_J1)
println("Mean abs errors for J=2.0: ", mean_J2)
println("Std abs errors for J=2.0: ", std_J2)

In [ ]:
mean_J1 =  [0.005004537364985043, 0.002530176749222324, 0.0027611126970210425, 0.001735685372116627, 0.0013517717779488042, 0.0011102406571519314, 0.00029257775484133053, 0.0004758642160053561] #error for J1
std_J1 =  [0.0021769224914989904, 0.0016819097300516507, 0.002084529102467281, 0.0008829649961288896, 0.0009431131198510869, 0.0006063750964076574, 0.0002403920672796746, 0.00046477694837289185] #error bars for J1
mean_J2 =  [0.06130598752316756, 0.03761695720368483, 0.04030916720933504, 0.020648498361473245, 0.015388615176824726, 0.012889425498476981, 0.008897651995752409, 0.006717418046892187] #error for J2
std_J2 =  [0.042252082765448, 0.02253689498747899, 0.03555865494413257, 0.017687398697120216, 0.009871507324690002, 0.009409828148682261, 0.0077186759982574345, 0.0038629900053143754] #error bars for J2